In [5]:
import numpy as np
import quantopian.optimize as opt
from quantopian.pipeline import Pipeline
from quantopian.pipeline.factors import SimpleMovingAverage

from quantopian.pipeline.filters import QTradableStocksUS
from quantopian.pipeline.experimental import risk_loading_pipeline

from quantopian.pipeline.data.psychsignal import stocktwits
from quantopian.pipeline.data import Fundamentals
#from quantopian.pipeline.factors import Returns
#import quantopian.pipeline.data.factset.estimates as fe 
from quantopian.pipeline.data import USEquityPricing  
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.research import run_pipeline

from alphalens.tears import create_information_tear_sheet
from alphalens.utils import get_clean_factor_and_forward_returns
import alphalens
import random as rd

# Constraint Parameters
MAX_GROSS_LEVERAGE = 1.0
TOTAL_POSITIONS = 300

# Here we define the maximum position size that can be held for any
# given stock. If you have a different idea of what these maximum
# sizes should be, feel free to change them. Keep in mind that the
# optimizer needs some leeway in order to operate. Namely, if your
# maximum is too small, the optimizer may be overly-constrained.
MAX_SHORT_POSITION_SIZE = 2.0 / TOTAL_POSITIONS
MAX_LONG_POSITION_SIZE = 2.0 / TOTAL_POSITIONS
global contadorSol
contadorSol=0
print("X")

X


In [6]:
def make_pipeline(PesoFactor):

    # The factors we create here are based on fundamentals data and a moving
    # average of sentiment data
    value = Fundamentals.ebit.latest / Fundamentals.enterprise_value.latest
    quality = Fundamentals.roe.latest
    sentiment_score = SimpleMovingAverage(inputs=[stocktwits.bull_minus_bear],window_length=3,)
    P1=Fundamentals.growth_score.latest
    P2=Fundamentals.value_score.latest
    P3=Fundamentals.style_score.latest
    P4 =AverageDollarVolume(window_length=5)
    P5 = Fundamentals.size_score.latest
    P6=Fundamentals.nace.latest
   
    #financial_health_grade.
    
    universe = QTradableStocksUS()
    
    value_winsorized = value.winsorize(min_percentile=0.05, max_percentile=0.95)
    quality_winsorized = quality.winsorize(min_percentile=0.05, max_percentile=0.95)
    sentiment_score_winsorized =                 sentiment_score.winsorize(min_percentile=0.05,max_percentile=0.95)
    P1_winsorized=P1.winsorize(min_percentile=0.05, max_percentile=0.95)
    P2_winsorized=P2.winsorize(min_percentile=0.05, max_percentile=0.95)
    P3_winsorized=P3.winsorize(min_percentile=0.05, max_percentile=0.95)
    P4_winsorized=P4.winsorize(min_percentile=0.00, max_percentile=0.90)
    P5_winsorized=P5.winsorize(min_percentile=0.00, max_percentile=0.9)
    P6_winsorized=P6.winsorize(min_percentile=0.05, max_percentile=0.9)
    
   
    FactorC=np.array([value_winsorized.zscore(),quality_winsorized.zscore(),
                      sentiment_score_winsorized.zscore(),P1_winsorized.zscore(),
                      P2_winsorized.zscore(),P3_winsorized.zscore(),P4_winsorized.zscore()
                      ,P5_winsorized.zscore(),P6_winsorized.zscore()])
    
    combined_factor = ((FactorC*PesoFactor).sum())
   
    longs = combined_factor.top(TOTAL_POSITIONS//2, mask=universe)
    shorts = combined_factor.bottom(TOTAL_POSITIONS//2, mask=universe)

    long_short_screen = (longs | shorts)

    # Create pipeline
    pipe = Pipeline(
        columns={
            #'longs': longs,
            #'shorts': shorts,
            'combined_factor': combined_factor
        },
        screen=long_short_screen
    )
    return pipe


In [11]:
def crer_poblacion_ini(tam_pob,num_fact):
    poblacion=np.zeros([tam_pob,num_fact+1])
    coeficientes=np.linspace(-5.0, 5.0, num=tam_pob)
    for i in range(0,num_fact):
        rd.shuffle(coeficientes)
        poblacion[:,i]=coeficientes
    return poblacion
def seleccion(poblacion,tam_pob,num_fact):
    a=np.random.randint(tam_pob, size=4)
    b=np.random.rand(4)
    sel=np.zeros([2,num_fact+1])
    #Selecion por el metodo del torneo estocastico
    if poblacion[a[0],num_fact]==poblacion[a[1],num_fact]:
        if b[0]>=b[1]:
            sel[0,:]=poblacion[a[0],:]
        else:
            sel[0,:]=poblacion[a[1],:]
    
    elif poblacion[a[0],num_fact]<poblacion[a[1],num_fact]:
        sel[0,:]=poblacion[a[0],:]
    else:
        sel[0,:]=poblacion[a[1],:]
        
    if poblacion[a[2],num_fact]==poblacion[a[3],num_fact]:
        if b[2]>=b[3]:
            sel[1,:]=poblacion[a[2],:]
        else:
            sel[1,:]=poblacion[a[3],:]
    
    elif poblacion[a[2],num_fact]<poblacion[a[3],num_fact]:
        sel[1,:]=poblacion[a[2],:]
    else:
        sel[1,:]=poblacion[a[3],:]

    return sel
def Cruzamiento(padres):
    p_H1=np.arange(padres.shape[1])
    rd.shuffle(p_H1)
    p_H2=np.arange(padres.shape[1])
    rd.shuffle(p_H2)
    hijos=np.zeros([padres.shape[0],padres.shape[1]])
    for i in range(0,padres.shape[1]):
        if i<=3:
            hijos[0,p_H1[i]]=padres[0,p_H1[i]]
            hijos[1,p_H2[i]]=padres[1,p_H2[i]]
        else:
            hijos[0,p_H1[i]]=padres[1,p_H1[i]]
            hijos[1,p_H2[i]]=padres[0,p_H2[i]]
    return hijos
def mutacion(hijos,num_fact,pm):
    for i in range(0,hijos.shape[0]):
        if np.random.rand()<pm: 
            pos=np.random.randint(num_fact)
            hijos[i,pos]=10*np.random.rand()-5         
def fitness(Individuo):
    factor_data = run_pipeline(make_pipeline(Individuo), '2019-08-06', '2020-02-06')
    pricing_data = get_pricing(factor_data.index.levels[1], '2019-08-06', '2020-02-06', fields='open_price')
    merged_data = get_clean_factor_and_forward_returns(factor_data, pricing_data)
    x=np.array(alphalens.performance.mean_information_coefficient(merged_data))
    
    
    print(" Valor del factor: "+ str(x[1]+x[0]))
    return x[1]+x[0]

In [ ]:
#=======================ALGORITMO GENETICO===============================
##----------Parametros--------------
tam_pob,num_fact=160,9
pc,pm=0.6,0.01
nGeneraciones=120
nGenSinMejora=90
##----------------------------------

contGSM=0
pob=crer_poblacion_ini(tam_pob,num_fact)
for p in range(0,pob.shape[0]):
    pob[p,pob.shape[1]-1]=fitness(pob[p,:pob.shape[1]-1])

for g in range(0,nGeneraciones):
    padres=seleccion(pob,tam_pob,num_fact)
    Hijos=np.zeros([2,num_fact+1])
    if np.random.rand() < pc:
        Hijos=Cruzamiento(padres)
    else:
        Hijos=padres
    mutacion(Hijos,num_fact,pm)
    pob=pob[np.argsort(pob[:,pob.shape[1]-1]*-1)]
    
    for h in range(0,Hijos.shape[0]):
        Hijos[h,Hijos.shape[1]-1]=fitness(Hijos[h,:Hijos.shape[1]-1])
        if Hijos[h,Hijos.shape[1]-1]>=pob[pob.shape[0]-1,pob.shape[1]-1]:
            pob[pob.shape[0]-1,:]=Hijos[h,:]
            if Hijos[h,Hijos.shape[1]-1]==pob[pob.shape[0]-1,pob.shape[1]-1]:
                 contGSM=contGSM+1
            else:
                contGSM=0
        else:
            contGSM=contGSM+1
    
    if contGSM>=nGenSinMejora:
        break
print(pob[0:5,:])
#========================================================================

Dropped 7.9% entries from factor data: 7.9% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: -0.0134783802496


Dropped 7.9% entries from factor data: 7.9% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: -0.00406809380548


Dropped 7.9% entries from factor data: 7.9% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: 0.0418969168404


Dropped 7.9% entries from factor data: 7.9% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: -0.00471237873408


Dropped 7.9% entries from factor data: 7.9% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: -0.00223257263284


...

Dropped 7.8% entries from factor data: 7.8% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: -0.0190847977385


Dropped 7.9% entries from factor data: 7.9% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: -0.0231441217321


Dropped 7.8% entries from factor data: 7.8% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: 0.0392107970699


Dropped 7.8% entries from factor data: 7.8% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: 0.0341484825583


Dropped 7.9% entries from factor data: 7.9% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
 Valor del factor: 0.000508711482795
